In [ ]:
import pennylane as qml
from pennylane import numpy as np
import itertools

# Create Hamiltonians

In [ ]:
def create_H_Heisenberg(Jx, Jz, l):
    '''
    Jx(int): X - interaction term in Heisenberg model 
    Jz(int): Z - interaction term in Heisenberg model
    l(int): Number of sites in Ising chain

    Creates a pennylane Hamiltonian representation of an Heisenberg chain with periodic boundary conditions
    '''
    coefficient_set = []
    gate_set = []

    for i in range(l-1):
        X_gate = 'I'*i + 'XX' + (l-i-2)*'I'
        gate_set.append(X_gate)
        coefficient_set.append(-Jx)

        Z_gate = 'I'*i + 'ZZ' + (l-i-2)*'I'
        gate_set.append(Z_gate)
        coefficient_set.append(-Jz)

    gate_set.append('Z' + 'I'*(l-2) + 'Z')
    coefficient_set.append(-Jz)

    gate_set.append('X' + 'I'*(l-2) + 'X')
    coefficient_set.append(-Jx)
        
    obs = [qml.pauli.string_to_pauli_word(pstrs) for pstrs in gate_set]
    ham = qml.Hamiltonian(coefficient_set, obs)
    return ham

def create_H_Ising(J, h, l):
    '''
    J(int): Interaction  term in Ising model 
    h(int): External magnetic fiels in Ising model
    l(int): Number of sites in Ising chain

    Creates a pennylane Hamiltonian representation of an Ising chain with periodic boundary conditions
    '''
    coefficient_set = []
    gate_set = []

    for i in range(l):
        X_gate = 'I'*i + 'X' + (l-i-1)*'I'
        gate_set.append(X_gate)
        coefficient_set.append(-h)
        if i<l-1:
            Z_gate = 'I'*i + 'ZZ' + (l-i-2)*'I'
            gate_set.append(Z_gate)
            coefficient_set.append(-J)

    gate_set.append('Z' + 'I'*(l-2) + 'Z')
    coefficient_set.append(-J)
        
    obs = [qml.pauli.string_to_pauli_word(pstrs) for pstrs in gate_set]
    ham = qml.Hamiltonian(coefficient_set, obs)
    return ham

# Find ground state and 1st excited state

In [ ]:
def eig_sort (w, v):
    '''
    w(np.array(float)): A vector containing the eigenvalues of a matrix
    v(np.array(np.array(float))): A vector containing the eigenvector of a matrix

    Returns 2 lists containing the eigenvalues and eigenvectors sorted in ascending order of the eigenvalue
    '''
    W = []
    V = []
    ind = np.argsort(w)
    for i in range(0, len(w)):
        W.append(w[ind[i]])
        V.append(v.T[ind[i]])
    return np.array(W),np.array(V).T

def ground_and_ext_states(params, l, model):
    '''
    Jx(int): X - interaction term in Heisenberg model 
    Jz(int): Z - interaction term in Heisenberg model
    l(int): Numbe of sites in Ising chain

    Find the ground state and the 1st excited state eigenvectors for an Hesienberg Hamiltonian
    '''

    if model == 'Ising':
        J = params[0]
        h = params[1]
        ham = create_H_Ising(J, h, l)
    elif model == 'Heisenberg':
        Jx = params[0]
        Jy = params[1]
        ham = create_H_Heisenberg(Jx, Jy, l)
    else:
        assert 'Incorrect Hamiltonian model'

    ham_matrix = qml.matrix(ham)
    val,vec = np.linalg.eig(ham_matrix)
    val,vec = eig_sort(val,vec)    

    ground = vec.T[0]
    ext = vec.T[1]
    return ground, ext

# Pauli strings functions

In [ ]:
def pauli_strings(n):
    '''
    n(int): Number of qubits

    Creates all possible combinations of pauli strings for n qubits
    '''

    paulis = ['I', 'X', 'Y', 'Z']
    combinations = itertools.product(paulis, repeat=n)
    return [''.join(comb) for comb in combinations]

def convert_str_to_matrix(pstr):
    '''
    pstr(str): A Pauli string

    Converts a Pauli string in a matrix
    '''
    for i,letter in enumerate(pstr):
        if i==0:
            if letter == 'I': 
                An = np.eye(2)
            elif letter == 'X':
                An = qml.PauliX.compute_matrix()
            elif letter == 'Y':
                An = qml.PauliY.compute_matrix()
            elif letter == 'Z':
                An = qml.PauliZ.compute_matrix()
        else:
            if letter == 'I': 
                An = np.kron(An, np.eye(2))
            elif letter == 'X':
                An = np.kron(An, qml.PauliX.compute_matrix())
            elif letter == 'Y':
                An = np.kron(An, qml.PauliY.compute_matrix())
            elif letter == 'Z':
                An = np.kron(An, qml.PauliZ.compute_matrix())
    return An

# Find Observables

This section finds the observables with the highest energy gap for both Ising and Heisenberg models

## Ising

In [ ]:
J = 0.1
h = 0.1

for n_qubits in range(3,8):
    print(f'N_QUBITS = {n_qubits}')
    pauli_str = pauli_strings(n_qubits)
    ground, ext = ground_and_ext_states([J, h], n_qubits, model='Ising')

    vals = []
    str_vals = []
    couples = []
    for pstr in pauli_str:
        mtx = convert_str_to_matrix(pstr)
        obs_01 = np.dot(np.conj(ground), np.dot(mtx, ext))
        couples.append((abs(obs_01), pstr))
    
    max_values = sorted(couples, reverse=True, key=lambda x: x[0])
    
    for i in enumerate(max_values[:5*n_qubits]):
        print(max_values[i[0]][1], max_values[i[0]][0])

    print('----------------')